In [7]:
!pip install torch==1.9.0 --quiet
!pip install torchvision==0.10.0 --quiet
!pip install detecto --quiet
!pip install flask-ngrok --quiet
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz --quiet
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2J8RN9l5ZFgqSojAbd42thI8yvL_B6PBRuQwHYXBEvfik6Zx 

     |████████████████████████████████| 831.4 MB 14 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.0+cu116 requires torch==1.13.0, but you have torch 1.9.0 which is incompatible.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.9.0 which is incompatible.
     |████████████████████████████████| 22.1 MB 26.2 MB/s 
ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [8]:
import cv2
from detecto import core, utils, visualize
from detecto.visualize import show_labeled_image
from detecto.utils import read_image
from flask import Flask,request,jsonify
from flask_ngrok import run_with_ngrok
from google.colab import drive
import numpy as np
import os
import pandas as pd

In [ ]:
#Generate New_Images_List.csv
#df = pd.DataFrame(columns=['image id','image name','class'])
#df.to_csv('New_Images_List.csv',encoding='utf-8', index=False)

In [6]:
#synonyms = class_df.loc[class_df['Class Name']==class1,'Synonym'].iloc[0]

In [10]:
#classes_list.csv, New_images_list.csv, model weights in the same folder as this file. 
drive.mount("/content/drive")
global base_path
base_path="/content/drive/MyDrive/Augmentation/"
os.chdir(base_path)
global image_path
image_path=base_path+"Unannotated/"
os.makedirs(image_path,exist_ok=True)
global classes_list_df
classes_list_df=pd.read_csv('/content/Classes_list.csv')
global classes_list
classes_list=list(classes_list_df['filename'])
global images_list_df
df = pd.DataFrame(columns=['image id','image name','class'])
df.to_csv('New_Images_List.csv',encoding='utf-8', index=False)
images_list_df=pd.read_csv('New_Images_List.csv')
model = core.Model.load('model_weights_epochs_14_lr_0001_001.pth', classes_list) 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
#If we were to return as a string, how should we separate the classes? Here we are using ','
def predict(image_dir,image_name):
  #image is numpy array
  #image_name is name of the image
  image=read_image(image_dir+image_name)
  try:
    image_count=images_list_df['image id'].iloc[-1]+1
  except IndexError:
    image_count=1
  #cv2.imwrite(os.path.join(image_path ,"User input image "+str(image_count)+".jpg"), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
  #cv2.waitKey(0)
  #images_list_df.loc[len(images_list_df.index)] = [image_count, "User input image "+str(image_count)+".jpg",None] 
  images_list_df.loc[len(images_list_df.index)] = [image_count,image_name,None]
  images_list_df.to_csv('New_Images_List.csv',encoding='utf-8', index=False)
  #print(image_dir+image_name)
  predictions = model.predict(image)
  labels, boxes, scores = predictions
  #print(scores)
  class_in_image=list()
  class_in_image=[x for x in scores if x>0.6]
  class_in_image=list(set(labels[0:len(class_in_image)]))
  return class_in_image

In [19]:
def class_in_list(s,string):
  """
  Returns string in s. This is to see whether class name exists in list(class_name). 
  This written as a function so that we can apply it on the dataframe.
  """
  return string in s

In [15]:
image_path=base_path+"Unannotated/"

In [25]:
app = Flask(__name__)
run_with_ngrok(app)   

#The csv containing both class name and synonyms
class_df = pd.read_csv("/content/drive/MyDrive/Augmentation/Classes_List.csv")

@app.route("/predict",methods=['POST'])
def show_class():
    f = request.files['file'] 
    f.save(image_path+f.filename)  
    classes = predict(image_path,f.filename)
    synonyms=[]
    for class_ in classes:
      #class_df is the csv file containing the class name and the synonyms
      if class_ in class_df['Class Name']:
        synonyms.append(class_df.loc[class_df['Class Name']==class_,'Synonym'].iloc[0])
        #print("In class name")
      elif class_ not in list(class_df['Class Name']):
        #print("In synonym")
        if(class_df['Synonym'].apply(class_in_list,args=([class_])).any()):
          i=class_df['Synonym'].apply(class_in_list,args=([class_]))
          synonyms.append(class_df.loc[i]['Class Name'].iloc[0])
    data = {
            "Img_name" : f.filename,
            "Prediction" : classes,
            "Alias": synonyms
        }
    return jsonify(data)

@app.route("/wrongclass",methods=['POST'])
def add_class():
    f=request.get_data()
    text_file = open("wrongclass.txt", "w")
    n = text_file.write(f.decode())
    text_file.close()     
    with open('wrongclass.txt', 'r') as text:
      data = text.read().rstrip()
      images_list_df.loc[len(images_list_df.index),'class']=data.split(',')
      images_list_df.to_csv('New_Images_List.csv',encoding='utf-8', index=False)
    return "Success"
    
@app.route("/helloworld",methods=['POST'])
def return_hello_world():
  return "Hello World!"
    
if __name__ == "__main__":
  app.run()
  #app.run(host=HOST, port=PORT, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c8e2-34-143-243-110.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [03/Jan/2023 12:32:45] "POST /predict HTTP/1.1" 200 -
